# Model training

In this file we have the validation and training for the `Feature Based` algorithms we are developing. Specifically, you can find:

- Cross Validation splits
- Feature Reduction / Feature selection ( by groups )
- Models (SVM and RandomForest)

It's relevant to understand that the proposed pipeline tries to validate the `models` we are implementing and how the different subsets/techniques change the this performance.

## 0. Relevant functions:

Here we set the `cross_validation` idea:

In [3]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os

# define basic files
FEATURES_DIR = os.path.join('..', 'data', 'features')
TRAIN_CSV_FILE = os.path.join('..', 'data', 'training_set.csv')
TEST_CSV_FILE = os.path.join('..', 'data', 'testing_set.csv')

# load the train data label
train_file = pd.read_csv(TRAIN_CSV_FILE)
test_file = pd.read_csv(TEST_CSV_FILE)

def load_subset_features( data, subset ):
    """ We load the pre-stored features"""
    features = np.load( os.path.join( FEATURES_DIR, subset_name + '.npy'), allow_pickle=True )[()]
    return [ features[fname] for fname in data['file'] ]

def generate_group_based_splits( data, n_splits=5, random_state=0):
    '''
    Uses the 'tag' column from train_data and makes `n_plists` stratified splits.
    Stratified -> balanced according to the labels
    Parameters:
        data (dataframe): with `tag` columns for `patient_id`.
    Returns:
        generator: yields the train and test indices
    '''
    stratified_kfolds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    unique_tag_data = data.drop_duplicates(subset=['patient_id'], keep='first')
    return stratified_kfolds.split( unique_tag_data['patient_id'], unique_tag_data['label'] )

def get_fold_features( train_fold, valid_fold, subset_name):
    """ Load the extracted features, from the pre-stored files """
    features = np.load( os.path.join( FEATURES_DIR, subset_name + '.npy'), allow_pickle=True )[()]
    train_features = [ features[fname] for fname in train_fold['file'] ]
    valid_features = [ features[fname] for fname in valid_fold['file'] ]
    return train_features, valid_features

def get_raw_features( fold_data ):
    """ 
    Stack the features for a full-training on the features.
    We just need to concatenate the features for each sample.
    """
    fts_order = [ 'hu_bin', 'fourier_real', 'fourier_imag', 'stats', 'gabor', 'lbp', 'haralick' ]
    data_fts = [ np.concatenate([ sample[ftname] for ftname in fts_order] ) for sample in fold_data ]
    data_fts = np.stack( data_fts )
    data_fts[ np.isnan(data_fts) ] = 0
    return data_fts

def find_elbow(data):
    cumulative_variance = np.cumsum( data ) / np.sum(data)
    return np.argmax(cumulative_variance >= 0.95) + 1

def apply_group_pca( tfold_data, vfold_data ):
    """ Group the features by type, calculate the optimal PCA compression and transform each group """
    fts_geo, fts_int, fts_text = [ 'hu_bin', 'fourier_real', 'fourier_imag'], [ 'stats', 'gabor' ], ['lbp', 'haralick' ]
    
    compressed_tdata, compressed_vdata = [], []
    for fts_group in [ fts_geo, fts_int, fts_text ]:
        tdata_fts = [ np.concatenate([ sample[ftname] for ftname in fts_group] ) for sample in tfold_data ]
        vdata_fts = [ np.concatenate([ sample[ftname] for ftname in fts_group] ) for sample in vfold_data ]
        tdata_fts, vdata_fts = np.stack( tdata_fts ), np.stack( vdata_fts )
        tdata_fts[ np.isnan(tdata_fts) ] = 0
        vdata_fts[ np.isnan(vdata_fts) ] = 0
        
        tdata_norm = StandardScaler().fit_transform( tdata_fts )
        vdata_norm = StandardScaler().fit_transform( vdata_fts )
        
        # study PCA
        pca = PCA()
        pca_result = pca.fit_transform( tdata_norm )
        optimal_components = find_elbow(pca.explained_variance_ratio_)
        
        # apply PCA
        tdata_pca = PCA(n_components=optimal_components).fit_transform( tdata_norm )
        vdata_pca = PCA(n_components=optimal_components).fit_transform( vdata_norm )
        compressed_tdata.append(tdata_pca), compressed_vdata.append(vdata_pca)
    return np.concatenate( compressed_tdata, axis=1), np.concatenate( compressed_vdata, axis=1)

def apply_group_thresholding( tfold_data, vfold_data, thresh=0.1 ):
    """ 
    Group the features by type and remove, by threhsolding, the features with low-variance
    """
    fts_geo, fts_int, fts_text = [ 'hu_bin', 'fourier_real', 'fourier_imag'], [ 'stats', 'gabor' ], ['lbp', 'haralick' ]
    
    reduced_tdata, reduced_vdata = [], []
    for fts_group in [ fts_geo, fts_int, fts_text ]:
        tdata_fts = [ np.concatenate([ sample[ftname] for ftname in fts_group] ) for sample in tfold_data ]
        vdata_fts = [ np.concatenate([ sample[ftname] for ftname in fts_group] ) for sample in vfold_data ]
        tdata_fts, vdata_fts = np.stack( tdata_fts ), np.stack( vdata_fts )
        tdata_fts[ np.isnan(tdata_fts) ] = 0
        vdata_fts[ np.isnan(vdata_fts) ] = 0

        # study variance thresholding
        thresholding = VarianceThreshold(threshold=thresh)
        tdata_reduced = thresholding.fit_transform( tdata_fts )
        vdata_reduced = thresholding.transform( vdata_fts )
        reduced_tdata.append(tdata_reduced), reduced_vdata.append(vdata_reduced)
    return np.concatenate( reduced_tdata, axis=1), np.concatenate( reduced_vdata, axis=1)

# 1. Support Vector Machines:

In [4]:
from itertools import product

DATA_SUBSETS = [ 
    'small_without_noise_not_inverted', 'small_with_noise_inverted', 'medium_with_noise_inverted'
]

# gridSearch for SVM
param_grid = {
    'C': [1, 5, 10], 
    'gamma': ['scale', 0.01],
    'kernel': ['poly', 'rbf'],
    'subset': DATA_SUBSETS
}

combinations = list( product(*param_grid.values()) )

GridSearch for the optimal results:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from tqdm import tqdm

# make all the combinations for the experiments
svm_results = []
for pvalues in tqdm( combinations, desc='Testing combination' ):
    svm_c, svm_gamma, svm_kernel = pvalues[0], pvalues[1], pvalues[2]
    data_subset = pvalues[3]
    
    fold_generator = generate_group_based_splits( train_file, n_splits=3 )
    combination_results = { 'raw': [], 'pca': [], 'var': [] }
    for tfold, vfold in fold_generator:
        
        tfold_data, vfold_data = train_file.iloc[tfold], train_file.iloc[vfold]
        tfold_Y, vfold_Y = tfold_data['label'].values, vfold_data['label'].values
        tfold_fts, vfold_fts = get_fold_features( tfold_data, vfold_data, subset_name=data_subset )

        # raw features
        tfold_fts_raw = get_raw_features( tfold_fts )
        vfold_fts_raw = get_raw_features( vfold_fts )

        # pca/thresholding features
        tfold_fts_pca, vfold_fts_pca = apply_group_pca( tfold_fts, vfold_fts )
        tfold_fts_var, vfold_fts_var = apply_group_thresholding( tfold_fts, vfold_fts )
        
        ### Raw Testing
        clf = make_pipeline( StandardScaler(), SVC(C=svm_c, gamma=svm_gamma, kernel=svm_kernel, class_weight='balanced') )
        clf.fit( tfold_fts_raw, tfold_Y )
        vfold_raw_pred = clf.predict( vfold_fts_raw )

        raw_acc = clf.score( vfold_fts_raw, vfold_Y )
        raw_f1 = f1_score( vfold_Y, vfold_raw_pred, average=None )[ np.where(clf[1].classes_ == 'T')[0][0] ]
        raw_f1_macro = f1_score( vfold_Y, vfold_raw_pred, average='macro') # overall, unweighted metric
        combination_results['raw'].append( (raw_acc, raw_f1, raw_f1_macro) )
        
        ### Group-based PCA
        clf = make_pipeline( SVC(C=svm_c, gamma=svm_gamma, kernel=svm_kernel, class_weight='balanced') )
        clf.fit( tfold_fts_pca, tfold_Y )
        vfold_pca_pred = clf.predict( vfold_fts_pca )

        pca_acc = clf.score( vfold_fts_pca, vfold_Y )
        pca_f1 = f1_score( vfold_Y, vfold_pca_pred, average=None )[ np.where(clf[0].classes_ == 'T')[0][0] ]
        pca_f1_macro = f1_score(vfold_Y, vfold_pca_pred, average='macro')
        combination_results['pca'].append( (pca_acc, pca_f1, pca_f1_macro) )
        
        ### Group-based Variance Thrhesholding
        clf = make_pipeline( StandardScaler(), SVC(C=svm_c, gamma=svm_gamma, kernel=svm_kernel, class_weight='balanced') )
        clf.fit( tfold_fts_var, tfold_Y )
        vfold_var_pred = clf.predict( vfold_fts_var )

        var_acc = clf.score( vfold_fts_var, vfold_Y )
        var_f1 = f1_score( vfold_Y, vfold_var_pred, average=None )[ np.where(clf[1].classes_ == 'T')[0][0] ]
        var_f1_macro = f1_score(vfold_Y, vfold_var_pred, average='macro')
        combination_results['var'].append( (var_acc, var_f1, var_f1_macro) )
        
    for rtype in combination_results:
        svm_results.append({
            'C': svm_c, 'Gamma': svm_gamma, 'Kernel': svm_kernel,
            'subset': data_subset,
            'Ft_type': rtype, 
            'acc': np.median( [ v[0] for v in combination_results[rtype] ] ),
            'f1_tub': np.median( [ v[1] for v in combination_results[rtype] ] ),
            'f1_macro': np.median( [ v[2] for v in combination_results[rtype] ] )
        })
    
svm_results = pd.DataFrame.from_records(svm_results)
svm_results.to_csv('svm_results.csv', index=False)

Testing combination:  89%|█████████████████████████████████████▎    | 32/36 [56:25<06:25, 96.28s/it]

In [1]:
svm_results.sort_values(by=['f1_tub', 'f1_macro'], ascending=False).head(n=10)

NameError: name 'svm_results' is not defined

# 2. Random Forest:

In [ ]:
from itertools import product

DATA_SUBSETS = [ 
    'small_without_noise_not_inverted', 'small_with_noise_inverted',
    'medium_without_noise_not_inverted', 'medium_with_noise_inverted', 
    # 'small_with_noise_not_inverted', 'small_without_noise_inverted'
]

# gridSearch for RandomForest
param_grid = {
    'n_estimators': [50, 100, 150], 
    'subset': DATA_SUBSETS
}

combinations = list( product(*param_grid.values()) )

Perform the gridseach over the results:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from tqdm import tqdm

# make all the combinations for the experiments
rf_results = []
for pvalues in tqdm( combinations, desc='Testing combination' ):
    rf_estimators = pvalues[0]
    data_subset = pvalues[1]
    
    fold_generator = generate_group_based_splits( train_file, n_splits=4 )
    combination_results = { 'raw': [], 'pca': [], 'var': [] }
    for tfold, vfold in fold_generator:
        
        tfold_data, vfold_data = train_file.iloc[tfold], train_file.iloc[vfold]
        tfold_Y, vfold_Y = tfold_data['label'].values, vfold_data['label'].values
        tfold_fts, vfold_fts = get_fold_features( tfold_data, vfold_data, subset_name=data_subset )

        # raw features
        tfold_fts_raw = get_raw_features( tfold_fts )
        vfold_fts_raw = get_raw_features( vfold_fts )

        # pca/thresholding features
        tfold_fts_pca, vfold_fts_pca = apply_group_pca( tfold_fts, vfold_fts )
        tfold_fts_var, vfold_fts_var = apply_group_thresholding( tfold_fts, vfold_fts )
        
        ### Raw Testing
        clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=rf_estimators, class_weight='balanced'))
        clf.fit( tfold_fts_raw, tfold_Y )
        vfold_raw_pred = clf.predict( vfold_fts_raw )

        raw_acc = clf.score( vfold_fts_raw, vfold_Y )
        raw_f1 = f1_score( vfold_Y, vfold_raw_pred, average=None )[ np.where(clf[1].classes_ == 'T')[0][0] ]
        raw_f1_macro = f1_score( vfold_Y, vfold_raw_pred, average='macro')
        combination_results['raw'].append( (raw_acc, raw_f1, raw_f1_macro) )
        
        ### Group-based PCA
        clf = make_pipeline( RandomForestClassifier(n_estimators=rf_estimators, class_weight='balanced'))
        clf.fit( tfold_fts_pca, tfold_Y )
        vfold_pca_pred = clf.predict( vfold_fts_pca )

        pca_acc = clf.score( vfold_fts_pca, vfold_Y )
        pca_f1 = f1_score( vfold_Y, vfold_pca_pred, average=None )[ np.where(clf[0].classes_ == 'T')[0][0] ]
        pca_f1_macro = f1_score(vfold_Y, vfold_pca_pred, average='macro')
        combination_results['pca'].append( (pca_acc, pca_f1, pca_f1_macro) )
        
        ### Group-based Variance Thrhesholding
        clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=rf_estimators, class_weight='balanced'))
        clf.fit( tfold_fts_var, tfold_Y )
        vfold_var_pred = clf.predict( vfold_fts_var )

        var_acc = clf.score( vfold_fts_var, vfold_Y )
        var_f1 = f1_score( vfold_Y, vfold_var_pred, average=None )[ np.where(clf[1].classes_ == 'T')[0][0] ]
        var_f1_macro = f1_score(vfold_Y, vfold_var_pred, average='macro')
        combination_results['var'].append( (var_acc, var_f1, var_f1_macro) )
        
    for rtype in combination_results:
        rf_results.append({
            'N_Estimators': rf_estimators,
            'Ft_type': rtype, 
            'acc': np.median( [ v[0] for v in combination_results[rtype] ] ),
            'f1': np.median( [ v[1] for v in combination_results[rtype] ] ),
            'f1_macro': np.median( [ v[2] for v in combination_results[rtype] ] ),
            'Subset': data_subset,
        })
    
rf_results = pd.DataFrame.from_records(rf_results)
rf_results.to_csv('rf_results.csv', index=False)
rf_results